In [1]:
import numpy as np
import pandas as pd
from keras import models
from keras import layers
from keras import optimizers
from numpy.random import seed
import tensorflow

seed(1907)
tensorflow.random.set_seed(2)


from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split 
from sklearn import metrics

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


import seaborn as sns

C:\Users\okana\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## FLOW:

I have the Call Details dataset with the target variable Churn flag. My aim is to construct models to accurately predict whether the call owner is going to churn or not. 

To do that, I will start with more "conventional" machine learning algorithms, then I will try my chances with neural networks. I will be focusing on optimizing the neural network to get the best possible result from the test data. I will be checking accuracy on the test data, but with imbalanced datasets, accuracy by itself might be misleading. Therefore, in parallel, I will be checking ROC AUC and F-1 scores on the test data, as well.

In [42]:
df = pd.read_csv('Call Details-Data.csv')
df['CHURN']= df['Churn']*1
df['CHURN'] = df['CHURN'].astype('int64')
df.drop(['Churn','Phone Number'],inplace=True,axis=1)


In [43]:
X = df.copy().drop('CHURN',axis=1)
y = df['CHURN']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)

In [ ]:
### Target Distribution:

df["CHURN"].count_values()

#### Other Model Trials:

In [263]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [264]:
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred1 = clf.predict(X_test)
y_pred_prob = clf.predict_proba(X_test)[:,1]

print(confusion_matrix(y_test, y_pred1))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred1))
print ("AUC Score:", roc_auc_score(y_test, y_pred1))
print ("AUC Score prob:", roc_auc_score(y_test, y_pred_prob))
print ("Precision:", precision_score(y_test, y_pred1))
print ("F1 Score:", f1_score(y_test, y_pred1))


[[26739   538]
 [  661  2415]]
Accuracy: 0.960498138569499
AUC Score: 0.8826934782601441
AUC Score prob: 0.8962798661976421
Precision: 0.8178123941754148
F1 Score: 0.80112788190413


In [265]:
rfc = RandomForestClassifier(n_estimators=100, max_features=10)

rfc = rfc.fit(X_train,y_train)

#Predict the response for test dataset
y_pred1 = rfc.predict(X_test)

y_pred_prob = rfc.predict_proba(X_test)[:,1]

print(confusion_matrix(y_test, y_pred1))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred1))
print ("AUC Score:", roc_auc_score(y_test, y_pred1))
print ("AUC Score prob:", roc_auc_score(y_test, y_pred_prob))
print ("Precision:", precision_score(y_test, y_pred1))
print ("F1 Score:", f1_score(y_test, y_pred1))


[[26910   367]
 [  628  2448]]
Accuracy: 0.9672190557770237
AUC Score: 0.8911920964198488
AUC Score prob: 0.9707196560661934
Precision: 0.8696269982238011
F1 Score: 0.8310982855202852


### Explanation:

- Churn column was boolean and it is converted to a simple 0-1 binary flag. 
- Grouping by the phone number, the number of records for that phone number 

### To Start:

- We are faced to a binary classification problem. Whilst predicting 0 and 1, binary_crossentropy is the most suitable cost function to avoid harsh cost changes.

- As the last layer activation function, I am going to start with the sigmoid. Later on, it will be replaced by softmax in the optimal selected models to compare.

- Activation functions, Optimizers and hyperparameters are going to be tried with diversity to observe their effect on the machine learning process.

In [72]:
## To avoid coding the same thing multiple times

def nn_model(network, num_epoch, size):
    
    network.fit(X_train, y_train, epochs = num_epoch, batch_size = size)
    nn_pred = network.predict_classes(X_test)
    nn_pred_prob= network.predict(X_test)


    print(confusion_matrix(y_test, nn_pred))
    print("Accuracy:",metrics.accuracy_score(y_test, nn_pred))
    print ("AUC Score:", roc_auc_score(y_test, nn_pred))
    print ("AUC Score prob:", roc_auc_score(y_test, nn_pred_prob))
    print ("F1 Score:", f1_score(y_test, nn_pred))    
    
    return metrics.accuracy_score(y_test, nn_pred),roc_auc_score(y_test, nn_pred_prob),f1_score(y_test, nn_pred)

## Optimizer: Adam

#### I have selected Adam as the optimizer for my initial trial:

In [54]:
network = models.Sequential()
network.add(layers.Dense(120, input_dim=15, activation='relu'))
network.add(layers.Dense(100, activation='relu'))
network.add(layers.Dense(1, activation='sigmoid'))
opt = optimizers.Adam(learning_rate=0.0001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-07)
network.compile(optimizer=opt,
loss='binary_crossentropy',
metrics=['accuracy'])

nn_model(network, 20, 20)

Epoch 1/20
3542/3542 [==============================] - 3s 816us/step - loss: 3.0312 - accuracy: 0.8379
Epoch 2/20
3542/3542 [==============================] - 3s 788us/step - loss: 1.9185 - accuracy: 0.8491
Epoch 3/20
3542/3542 [==============================] - 3s 791us/step - loss: 1.6999 - accuracy: 0.8555
Epoch 4/20
3542/3542 [==============================] - 3s 788us/step - loss: 1.4853 - accuracy: 0.8586
Epoch 5/20
3542/3542 [==============================] - 3s 796us/step - loss: 1.3500 - accuracy: 0.8653
Epoch 6/20
3542/3542 [==============================] - 3s 798us/step - loss: 1.3405 - accuracy: 0.8692
Epoch 7/20
3542/3542 [==============================] - 3s 792us/step - loss: 1.3553 - accuracy: 0.8700
Epoch 8/20
3542/3542 [==============================] - 3s 794us/step - loss: 1.2373 - accuracy: 0.8720
Epoch 9/20
3542/3542 [==============================] - 3s 806us/step - loss: 1.1215 - accuracy: 0.8783
Epoch 10/20
3542/3542 [==============================] - 3s 801u

##### Increasing number of neurons did not work out:

In [55]:
network = models.Sequential()
network.add(layers.Dense(200, input_dim=15, activation='relu'))
network.add(layers.Dense(120, activation='relu'))
network.add(layers.Dense(1, activation='sigmoid'))
opt = optimizers.Adam(learning_rate=0.0001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-07)
network.compile(optimizer=opt,
loss='binary_crossentropy',
metrics=['accuracy'])

nn_model(network, 20, 20)

Epoch 1/20
3542/3542 [==============================] - 3s 913us/step - loss: 4.1427 - accuracy: 0.8352
Epoch 2/20
3542/3542 [==============================] - 3s 935us/step - loss: 2.8270 - accuracy: 0.8411
Epoch 3/20
3542/3542 [==============================] - 3s 914us/step - loss: 2.5386 - accuracy: 0.8442
Epoch 4/20
3542/3542 [==============================] - 3s 909us/step - loss: 2.3082 - accuracy: 0.8507
Epoch 5/20
3542/3542 [==============================] - 3s 913us/step - loss: 1.9932 - accuracy: 0.8548
Epoch 6/20
3542/3542 [==============================] - 3s 914us/step - loss: 1.8901 - accuracy: 0.8590
Epoch 7/20
3542/3542 [==============================] - 3s 911us/step - loss: 1.8493 - accuracy: 0.8603
Epoch 8/20
3542/3542 [==============================] - 3s 926us/step - loss: 1.7553 - accuracy: 0.8641
Epoch 9/20
3542/3542 [==============================] - 3s 923us/step - loss: 1.5323 - accuracy: 0.8666
Epoch 10/20
3542/3542 [==============================] - 3s 920u

#### Instead, the positive effect of increasing the number of layers can be seen by checking the roc_auc_proba and the final loss value. Besides, the rate of decrease in the loss value at the final epoch is significant. For the sake of time and computation power, I will optimize other parts first, knowing that optimization of batch and epoch, and then I will fine-tune those too.

In [68]:
network = models.Sequential()
network.add(layers.Dense(120, input_dim=15, activation='relu'))
network.add(layers.Dense(150, activation='relu'))
network.add(layers.Dense(100, activation='relu'))
network.add(layers.Dense(1, activation='sigmoid'))
opt = optimizers.Adam(learning_rate=0.0001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-07)
network.compile(optimizer=opt,
loss='binary_crossentropy',
metrics=['accuracy'])

nn_model(network, 20, 20)

Epoch 1/20
3542/3542 [==============================] - 4s 1ms/step - loss: 2.8113 - accuracy: 0.8414
Epoch 2/20
3542/3542 [==============================] - 4s 1ms/step - loss: 1.5681 - accuracy: 0.8633
Epoch 3/20
3542/3542 [==============================] - 4s 1ms/step - loss: 1.2190 - accuracy: 0.8739
Epoch 4/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.9218 - accuracy: 0.8823
Epoch 5/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.7828 - accuracy: 0.8859
Epoch 6/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.6788 - accuracy: 0.8886
Epoch 7/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.5904 - accuracy: 0.8897
Epoch 8/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.5131 - accuracy: 0.8905
Epoch 9/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.4629 - accuracy: 0.8914
Epoch 10/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.423

##### Instead of relu -> selu:

In [69]:
network = models.Sequential()
network.add(layers.Dense(120, input_dim=15, activation='selu'))
network.add(layers.Dense(150,  activation='selu'))
network.add(layers.Dense(100, activation='selu'))
network.add(layers.Dense(1, activation='sigmoid'))
opt = optimizers.Adam(learning_rate=0.0001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-07)
network.compile(optimizer=opt,
loss='binary_crossentropy',
metrics=['accuracy'])

nn_model(network, 20, 20)

Epoch 1/20
3542/3542 [==============================] - 4s 1ms/step - loss: 3.7955 - accuracy: 0.8321
Epoch 2/20
3542/3542 [==============================] - 4s 1ms/step - loss: 2.2409 - accuracy: 0.8485
Epoch 3/20
3542/3542 [==============================] - 4s 1ms/step - loss: 1.8645 - accuracy: 0.8541
Epoch 4/20
3542/3542 [==============================] - 4s 1ms/step - loss: 1.4915 - accuracy: 0.8640
Epoch 5/20
3542/3542 [==============================] - 4s 1ms/step - loss: 1.2066 - accuracy: 0.8717
Epoch 6/20
3542/3542 [==============================] - 4s 1ms/step - loss: 1.0199 - accuracy: 0.8791
Epoch 7/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.8479 - accuracy: 0.8827
Epoch 8/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.7339 - accuracy: 0.8860
Epoch 9/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.5784 - accuracy: 0.8897
Epoch 10/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.506

#### Let's try multiple parameters in a loop:

In [83]:
inp = [80,100,120,150]
l1 = [150,200,250,300]
l2 =[100,150,200,250]

compare = pd.DataFrame(columns=['i','j','k','accuracy','roc','f1'])
m=0
for i in inp:
    for j in l1:
        for k in l2:
            network = models.Sequential()
            network.add(layers.Dense(i, input_dim=15, activation='selu'))
            network.add(layers.Dense(j,  activation='selu'))
            network.add(layers.Dense(k, activation='selu'))
            network.add(layers.Dense(1, activation='sigmoid'))
            opt = optimizers.Adam(learning_rate=0.0001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-07)
            network.compile(optimizer=opt,
            loss='binary_crossentropy',
            metrics=['accuracy'])

            
            a = [i,j,k]+ list(nn_model(network, 20, 20))
            
            compare.loc[m] = a
            m += 1
            display(compare)

compare.to_excel('compare.xlsx')

Epoch 1/20
3542/3542 [==============================] - 4s 1ms/step - loss: 4.2877 - accuracy: 0.8329
Epoch 2/20
3542/3542 [==============================] - 4s 1ms/step - loss: 2.6326 - accuracy: 0.8423
Epoch 3/20
3542/3542 [==============================] - 4s 1ms/step - loss: 2.1849 - accuracy: 0.8497
Epoch 4/20
3542/3542 [==============================] - 4s 1ms/step - loss: 1.8854 - accuracy: 0.8579
Epoch 5/20
3542/3542 [==============================] - 4s 1ms/step - loss: 1.6021 - accuracy: 0.8641
Epoch 6/20
3542/3542 [==============================] - 4s 1ms/step - loss: 1.3673 - accuracy: 0.8742
Epoch 7/20
3542/3542 [==============================] - 4s 1ms/step - loss: 1.1259 - accuracy: 0.8785
Epoch 8/20
3542/3542 [==============================] - 6s 2ms/step - loss: 1.0071 - accuracy: 0.8826
Epoch 9/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.7938 - accuracy: 0.8867
Epoch 10/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.759

,i,j,k,accuracy,roc,f1
0,80.0,150.0,100.0,0.893553,0.693224,0.133548


Epoch 1/20
3542/3542 [==============================] - 5s 1ms/step - loss: 3.8040 - accuracy: 0.8320
Epoch 2/20
3542/3542 [==============================] - 5s 1ms/step - loss: 2.3862 - accuracy: 0.8483
Epoch 3/20
3542/3542 [==============================] - 5s 1ms/step - loss: 1.9580 - accuracy: 0.8557
Epoch 4/20
3542/3542 [==============================] - 5s 1ms/step - loss: 1.5283 - accuracy: 0.8680
Epoch 5/20
3542/3542 [==============================] - 5s 1ms/step - loss: 1.2587 - accuracy: 0.8754
Epoch 6/20
3542/3542 [==============================] - 5s 1ms/step - loss: 1.0723 - accuracy: 0.8799
Epoch 7/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.9046 - accuracy: 0.8832
Epoch 8/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.7367 - accuracy: 0.8877
Epoch 9/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.6459 - accuracy: 0.8896
Epoch 10/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.584

,i,j,k,accuracy,roc,f1
0,80.0,150.0,100.0,0.893553,0.693224,0.133548
1,80.0,150.0,150.0,0.893849,0.707529,0.111418


Epoch 1/20
3542/3542 [==============================] - 6s 2ms/step - loss: 3.3220 - accuracy: 0.8420
Epoch 2/20
3542/3542 [==============================] - 5s 2ms/step - loss: 2.1991 - accuracy: 0.8548
Epoch 3/20
3542/3542 [==============================] - 6s 2ms/step - loss: 1.7741 - accuracy: 0.8649
Epoch 4/20
3542/3542 [==============================] - 5s 2ms/step - loss: 1.2827 - accuracy: 0.8758
Epoch 5/20
3542/3542 [==============================] - 5s 2ms/step - loss: 1.0288 - accuracy: 0.8823
Epoch 6/20
3542/3542 [==============================] - 5s 2ms/step - loss: 0.8914 - accuracy: 0.8870
Epoch 7/20
3542/3542 [==============================] - 5s 2ms/step - loss: 0.7425 - accuracy: 0.8885
Epoch 8/20
3542/3542 [==============================] - 5s 2ms/step - loss: 0.6375 - accuracy: 0.8899
Epoch 9/20
3542/3542 [==============================] - 6s 2ms/step - loss: 0.5295 - accuracy: 0.8919
Epoch 10/20
3542/3542 [==============================] - 5s 2ms/step - loss: 0.492

...

,i,j,k,accuracy,roc,f1
0,80.0,150.0,100.0,0.893553,0.693224,0.133548
1,80.0,150.0,150.0,0.893849,0.707529,0.111418
2,80.0,150.0,200.0,0.897111,0.737695,0.170077
3,80.0,150.0,250.0,0.898890,0.760174,0.168968
4,80.0,200.0,100.0,0.893717,0.716424,0.128579
...,...,...,...,...,...,...
57,150.0,250.0,150.0,0.898363,0.787910,0.190926
58,150.0,250.0,200.0,0.898857,0.774579,0.163943
59,150.0,250.0,250.0,0.897901,0.772246,0.160390
60,150.0,300.0,100.0,0.895035,0.755075,0.151305


Epoch 1/20
3542/3542 [==============================] - 6s 2ms/step - loss: 4.5305 - accuracy: 0.8320
Epoch 2/20
3542/3542 [==============================] - 6s 2ms/step - loss: 2.3595 - accuracy: 0.8556
Epoch 3/20
3542/3542 [==============================] - 6s 2ms/step - loss: 1.4606 - accuracy: 0.8733
Epoch 4/20
3542/3542 [==============================] - 6s 2ms/step - loss: 0.9290 - accuracy: 0.8842
Epoch 5/20
3542/3542 [==============================] - 6s 2ms/step - loss: 0.6202 - accuracy: 0.8891
Epoch 6/20
3542/3542 [==============================] - 5s 2ms/step - loss: 0.4879 - accuracy: 0.8908
Epoch 7/20
3542/3542 [==============================] - 5s 2ms/step - loss: 0.4089 - accuracy: 0.8931
Epoch 8/20
3542/3542 [==============================] - 5s 2ms/step - loss: 0.3615 - accuracy: 0.8946
Epoch 9/20
3542/3542 [==============================] - 6s 2ms/step - loss: 0.3372 - accuracy: 0.8960
Epoch 10/20
3542/3542 [==============================] - 5s 2ms/step - loss: 0.328

,i,j,k,accuracy,roc,f1
0,80.0,150.0,100.0,0.893553,0.693224,0.133548
1,80.0,150.0,150.0,0.893849,0.707529,0.111418
2,80.0,150.0,200.0,0.897111,0.737695,0.170077
3,80.0,150.0,250.0,0.898890,0.760174,0.168968
4,80.0,200.0,100.0,0.893717,0.716424,0.128579
...,...,...,...,...,...,...
58,150.0,250.0,200.0,0.898857,0.774579,0.163943
59,150.0,250.0,250.0,0.897901,0.772246,0.160390
60,150.0,300.0,100.0,0.895035,0.755075,0.151305
61,150.0,300.0,150.0,0.900438,0.796875,0.205991


Epoch 1/20
3542/3542 [==============================] - 6s 2ms/step - loss: 4.1204 - accuracy: 0.8380
Epoch 2/20
3542/3542 [==============================] - 6s 2ms/step - loss: 2.0733 - accuracy: 0.8585
Epoch 3/20
3542/3542 [==============================] - 6s 2ms/step - loss: 1.2397 - accuracy: 0.8752
Epoch 4/20
3542/3542 [==============================] - 6s 2ms/step - loss: 0.7608 - accuracy: 0.8867
Epoch 5/20
3542/3542 [==============================] - 6s 2ms/step - loss: 0.5533 - accuracy: 0.8896
Epoch 6/20
3542/3542 [==============================] - 6s 2ms/step - loss: 0.4498 - accuracy: 0.8921
Epoch 7/20
3542/3542 [==============================] - 6s 2ms/step - loss: 0.3996 - accuracy: 0.8932
Epoch 8/20
3542/3542 [==============================] - 6s 2ms/step - loss: 0.3566 - accuracy: 0.8944
Epoch 9/20
3542/3542 [==============================] - 6s 2ms/step - loss: 0.3399 - accuracy: 0.8962
Epoch 10/20
3542/3542 [==============================] - 6s 2ms/step - loss: 0.328

,i,j,k,accuracy,roc,f1
0,80.0,150.0,100.0,0.893553,0.693224,0.133548
1,80.0,150.0,150.0,0.893849,0.707529,0.111418
2,80.0,150.0,200.0,0.897111,0.737695,0.170077
3,80.0,150.0,250.0,0.898890,0.760174,0.168968
4,80.0,200.0,100.0,0.893717,0.716424,0.128579
...,...,...,...,...,...,...
59,150.0,250.0,250.0,0.897901,0.772246,0.160390
60,150.0,300.0,100.0,0.895035,0.755075,0.151305
61,150.0,300.0,150.0,0.900438,0.796875,0.205991
62,150.0,300.0,200.0,0.897934,0.768065,0.162250


In [88]:
compare.sort_values(by='roc',ascending=False)[0:10]\
       .sort_values(by='f1',ascending=False)[0:3]\
       .sort_values(by='accuracy',ascending=False)

,i,j,k,accuracy,roc,f1
41,120.0,250.0,150.0,0.900241,0.795194,0.221994
47,120.0,300.0,250.0,0.899911,0.798087,0.207203
11,80.0,250.0,250.0,0.898725,0.784224,0.216216


#### i=120, j= 250, k=150 selected:

In [89]:
network = models.Sequential()
network.add(layers.Dense(120, input_dim=15, activation='selu'))
network.add(layers.Dense(250,  activation='selu'))
network.add(layers.Dense(150, activation='selu'))
network.add(layers.Dense(1, activation='sigmoid'))
opt = optimizers.Adam(learning_rate=0.0001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-07)
network.compile(optimizer=opt,
loss='binary_crossentropy',
metrics=['accuracy'])

nn_model(network, 20, 20)

Epoch 1/20
3542/3542 [==============================] - 5s 1ms/step - loss: 4.1696 - accuracy: 0.8383
Epoch 2/20
3542/3542 [==============================] - 5s 1ms/step - loss: 2.3418 - accuracy: 0.8559
Epoch 3/20
3542/3542 [==============================] - 5s 1ms/step - loss: 1.7734 - accuracy: 0.8630
Epoch 4/20
3542/3542 [==============================] - 5s 1ms/step - loss: 1.2501 - accuracy: 0.8756
Epoch 5/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.8945 - accuracy: 0.8842
Epoch 6/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.7254 - accuracy: 0.8882
Epoch 7/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.5873 - accuracy: 0.8894
Epoch 8/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.4740 - accuracy: 0.8915
Epoch 9/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.4013 - accuracy: 0.8937
Epoch 10/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.380

(0.8979013606562778, 0.7730362321476267, 0.1571933641555616)

## Optimizer: Adagrad

##### The same model and the performance of the model diminished. I will try to optimize Adagrad hyperparameters:

In [90]:
network = models.Sequential()
network.add(layers.Dense(120, input_dim=15, activation='selu'))
network.add(layers.Dense(250,activation='selu'))
network.add(layers.Dense(150, activation='selu'))
network.add(layers.Dense(1, activation='sigmoid'))
opt = optimizers.Adagrad(learning_rate=0.0001, epsilon = 1e-07)
network.compile(optimizer=opt,
loss='binary_crossentropy',
metrics=['accuracy'])

nn_model(network, 20, 20)

Epoch 1/20
3542/3542 [==============================] - 4s 1ms/step - loss: 2.6001 - accuracy: 0.8494
Epoch 2/20
3542/3542 [==============================] - 4s 1ms/step - loss: 1.6392 - accuracy: 0.8581
Epoch 3/20
3542/3542 [==============================] - 4s 1ms/step - loss: 1.4578 - accuracy: 0.8600
Epoch 4/20
3542/3542 [==============================] - 4s 1ms/step - loss: 1.3574 - accuracy: 0.8617
Epoch 5/20
3542/3542 [==============================] - 4s 1ms/step - loss: 1.2908 - accuracy: 0.8626
Epoch 6/20
3542/3542 [==============================] - 4s 1ms/step - loss: 1.2365 - accuracy: 0.8634
Epoch 7/20
3542/3542 [==============================] - 4s 1ms/step - loss: 1.1947 - accuracy: 0.8648
Epoch 8/20
3542/3542 [==============================] - 4s 1ms/step - loss: 1.1509 - accuracy: 0.8640
Epoch 9/20
3542/3542 [==============================] - 4s 1ms/step - loss: 1.1193 - accuracy: 0.8660
Epoch 10/20
3542/3542 [==============================] - 4s 1ms/step - loss: 1.087

(0.8649556880703719, 0.543420010220486, 0.12057498390903237)

##### with initial accumulator 0.1 ve learning rate 0.01 near-Adam performance. F1 beats:

In [91]:
network = models.Sequential()
network.add(layers.Dense(120, input_dim=15, activation='selu'))
network.add(layers.Dense(250, activation='selu'))
network.add(layers.Dense(150, activation='selu'))
network.add(layers.Dense(1, activation='sigmoid'))
opt = optimizers.Adagrad(learning_rate=0.01,initial_accumulator_value=0.1, epsilon = 1e-07)
network.compile(optimizer=opt,
loss='binary_crossentropy',
metrics=['accuracy'])

nn_model(network, 20, 20)

Epoch 1/20
3542/3542 [==============================] - 5s 1ms/step - loss: 2.1560 - accuracy: 0.8784
Epoch 2/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.5034 - accuracy: 0.8907
Epoch 3/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.4105 - accuracy: 0.8934
Epoch 4/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.3726 - accuracy: 0.8956
Epoch 5/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.3470 - accuracy: 0.8978
Epoch 6/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.3305 - accuracy: 0.8988
Epoch 7/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.3260 - accuracy: 0.8995
Epoch 8/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.3150 - accuracy: 0.9010
Epoch 9/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.3100 - accuracy: 0.9012
Epoch 10/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.303

(0.8977366322933482, 0.7553238069102646, 0.2358444116198917)

#### Learning rate 0.1 works better:

In [92]:
network = models.Sequential()
network.add(layers.Dense(120, input_dim=15, activation='selu'))
network.add(layers.Dense(250, input_dim=15, activation='selu'))
network.add(layers.Dense(150, activation='selu'))
network.add(layers.Dense(1, activation='sigmoid'))
opt = optimizers.Adagrad(learning_rate=0.001,initial_accumulator_value=0.1, epsilon = 1e-07)
network.compile(optimizer=opt,
loss='binary_crossentropy',
metrics=['accuracy'])

nn_model(network, 20, 20)

Epoch 1/20
3542/3542 [==============================] - 4s 1ms/step - loss: 2.2854 - accuracy: 0.8489
Epoch 2/20
3542/3542 [==============================] - 4s 1ms/step - loss: 1.1710 - accuracy: 0.8659
Epoch 3/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.9442 - accuracy: 0.8715
Epoch 4/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.8305 - accuracy: 0.8760
Epoch 5/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.7510 - accuracy: 0.8795
Epoch 6/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.6884 - accuracy: 0.8816
Epoch 7/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.6697 - accuracy: 0.8824
Epoch 8/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.6304 - accuracy: 0.8838
Epoch 9/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.5949 - accuracy: 0.8859
Epoch 10/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.588

(0.8569498896319968, 0.6290018210669406, 0.18992537313432836)

## Adadelta:

In [94]:
network = models.Sequential()
network.add(layers.Dense(120, input_dim=15, activation='selu'))
network.add(layers.Dense(250, input_dim=15, activation='selu'))
network.add(layers.Dense(150, activation='selu'))
network.add(layers.Dense(1, activation='sigmoid'))
opt = optimizers.Adadelta(learning_rate=0.001, epsilon = 1e-07)
network.compile(optimizer=opt,
loss='binary_crossentropy',
metrics=['accuracy'])

nn_model(network, 20, 20)

Epoch 1/20
3542/3542 [==============================] - 5s 1ms/step - loss: 7.1644 - accuracy: 0.8417
Epoch 2/20
3542/3542 [==============================] - 5s 1ms/step - loss: 4.1448 - accuracy: 0.8010
Epoch 3/20
3542/3542 [==============================] - 5s 1ms/step - loss: 3.3451 - accuracy: 0.8129
Epoch 4/20
3542/3542 [==============================] - 5s 1ms/step - loss: 2.9438 - accuracy: 0.8203
Epoch 5/20
3542/3542 [==============================] - 5s 1ms/step - loss: 2.6625 - accuracy: 0.8274
Epoch 6/20
3542/3542 [==============================] - 5s 1ms/step - loss: 2.4629 - accuracy: 0.8311
Epoch 7/20
3542/3542 [==============================] - 5s 1ms/step - loss: 2.2967 - accuracy: 0.8352
Epoch 8/20
3542/3542 [==============================] - 5s 1ms/step - loss: 2.1604 - accuracy: 0.8387
Epoch 9/20
3542/3542 [==============================] - 5s 1ms/step - loss: 2.0529 - accuracy: 0.8422
Epoch 10/20
3542/3542 [==============================] - 5s 1ms/step - loss: 1.963

(0.8554343886930452, 0.5332186548389152, 0.0892486508924865)

#### In the same cell, learning rates 0.1, 0.01, 0.001 and rho 0.9, 0.95, 0.99 are tried but the best performing one -- 0.01 / 0.95 -- cannot beat the model with Adam as optimizer:

In [100]:
network = models.Sequential()
network.add(layers.Dense(120, input_dim=15, activation='selu'))
network.add(layers.Dense(250, input_dim=15, activation='selu'))
network.add(layers.Dense(150, activation='selu'))
network.add(layers.Dense(1, activation='sigmoid'))
opt = optimizers.Adadelta(learning_rate=0.01, rho = 0.95, epsilon = 1e-07)
network.compile(optimizer=opt,
loss='binary_crossentropy',
metrics=['accuracy'])

nn_model(network, 20, 20)

Epoch 1/20
3542/3542 [==============================] - 5s 1ms/step - loss: 4.0845 - accuracy: 0.8446
Epoch 2/20
3542/3542 [==============================] - 5s 1ms/step - loss: 1.6760 - accuracy: 0.8551
Epoch 3/20
3542/3542 [==============================] - 5s 1ms/step - loss: 1.2808 - accuracy: 0.8630
Epoch 4/20
3542/3542 [==============================] - 5s 1ms/step - loss: 1.0821 - accuracy: 0.8676
Epoch 5/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.9572 - accuracy: 0.8701
Epoch 6/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.8842 - accuracy: 0.8736
Epoch 7/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.8228 - accuracy: 0.8752
Epoch 8/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.7829 - accuracy: 0.8762
Epoch 9/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.7364 - accuracy: 0.8792
Epoch 10/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.715

(0.8740816393766678, 0.6222717755796254, 0.16036906854130056)

## RMSprop:

#### Results and the rate of decrease in the loss function is significant in the first trial:

In [101]:
network = models.Sequential()
network.add(layers.Dense(120, input_dim=15, activation='selu'))
network.add(layers.Dense(250, input_dim=15, activation='selu'))
network.add(layers.Dense(150, activation='selu'))
network.add(layers.Dense(1, activation='sigmoid'))
opt = optimizers.RMSprop(learning_rate=0.0001,  epsilon = 1e-07)
network.compile(optimizer=opt,
loss='binary_crossentropy',
metrics=['accuracy'])

nn_model(network, 20, 20)

Epoch 1/20
3542/3542 [==============================] - 5s 1ms/step - loss: 2.8494 - accuracy: 0.8379
Epoch 2/20
3542/3542 [==============================] - 5s 1ms/step - loss: 1.8461 - accuracy: 0.8555
Epoch 3/20
3542/3542 [==============================] - 5s 1ms/step - loss: 1.4463 - accuracy: 0.8661
Epoch 4/20
3542/3542 [==============================] - 5s 1ms/step - loss: 1.2512 - accuracy: 0.8747
Epoch 5/20
3542/3542 [==============================] - 5s 1ms/step - loss: 1.1036 - accuracy: 0.8800
Epoch 6/20
3542/3542 [==============================] - 5s 1ms/step - loss: 1.0375 - accuracy: 0.8827
Epoch 7/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.9300 - accuracy: 0.8855
Epoch 8/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.8858 - accuracy: 0.8873
Epoch 9/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.7793 - accuracy: 0.8895
Epoch 10/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.740

(0.8707541264454914, 0.6715043592236629, 0.19296441061509978)

#### In the same cell, different parameters are tried and with some of them, I have achieved better scores. But, I am only testing with limited epochs and batch, so the change in loss function in the final epochs is important to me:

In [108]:
network = models.Sequential()
network.add(layers.Dense(120, input_dim=15, activation='selu'))
network.add(layers.Dense(250, input_dim=15, activation='selu'))
network.add(layers.Dense(150, activation='selu'))
network.add(layers.Dense(1, activation='sigmoid'))
opt = optimizers.RMSprop(learning_rate=0.00001, momentum=0.01, epsilon = 1e-07)
network.compile(optimizer=opt,
loss='binary_crossentropy',
metrics=['accuracy'])

nn_model(network, 20, 20)

Epoch 1/20
3542/3542 [==============================] - 5s 1ms/step - loss: 2.4217 - accuracy: 0.8451
Epoch 2/20
3542/3542 [==============================] - 5s 1ms/step - loss: 1.1822 - accuracy: 0.8656
Epoch 3/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.9849 - accuracy: 0.8729
Epoch 4/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.8954 - accuracy: 0.8761
Epoch 5/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.8221 - accuracy: 0.8791
Epoch 6/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.7772 - accuracy: 0.8798
Epoch 7/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.7515 - accuracy: 0.8810
Epoch 8/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.7389 - accuracy: 0.8819
Epoch 9/20
3542/3542 [==============================] - 5s 2ms/step - loss: 0.7082 - accuracy: 0.8846
Epoch 10/20
3542/3542 [==============================] - 5s 1ms/step - loss: 0.691

(0.8630448390603894, 0.6585209614965628, 0.21728488043682923)

## SGD:

Simply worse:

In [110]:
network = models.Sequential()
network.add(layers.Dense(120, input_dim=15, activation='selu'))
network.add(layers.Dense(250, input_dim=15, activation='selu'))
network.add(layers.Dense(150, activation='selu'))
network.add(layers.Dense(1, activation='sigmoid'))
opt = optimizers.SGD(learning_rate=0.0001)
network.compile(optimizer=opt,
loss='binary_crossentropy',
metrics=['accuracy'])

nn_model(network, 20, 20)

Epoch 1/20
3542/3542 [==============================] - 4s 1ms/step - loss: 2.3652 - accuracy: 0.8654
Epoch 2/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.4134 - accuracy: 0.8910
Epoch 3/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.3509 - accuracy: 0.8948
Epoch 4/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.3375 - accuracy: 0.8957
Epoch 5/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.3305 - accuracy: 0.8967
Epoch 6/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.3272 - accuracy: 0.8972
Epoch 7/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.3254 - accuracy: 0.8974
Epoch 8/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.3234 - accuracy: 0.8973
Epoch 9/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.3224 - accuracy: 0.8979
Epoch 10/20
3542/3542 [==============================] - 4s 1ms/step - loss: 0.321

(0.8936513688926959, 0.6644941461116913, 0.07560137457044674)

# FINAL SELECTION:

ADAM

In [113]:
network = models.Sequential()
network.add(layers.Dense(120, input_dim=15, activation='selu'))
network.add(layers.Dense(250,  activation='selu'))
network.add(layers.Dense(150, activation='selu'))
network.add(layers.Dense(1, activation='sigmoid'))
opt = optimizers.Adam(learning_rate=0.0001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-07)
network.compile(optimizer=opt,
loss='binary_crossentropy',
metrics=['accuracy'])

nn_model(network, 200, 5)

Epoch 1/200
14165/14165 [==============================] - 17s 1ms/step - loss: 4.6951 - accuracy: 0.8346
Epoch 2/200
14165/14165 [==============================] - 17s 1ms/step - loss: 1.2768 - accuracy: 0.8753
Epoch 3/200
14165/14165 [==============================] - 17s 1ms/step - loss: 0.5232 - accuracy: 0.8894
Epoch 4/200
14165/14165 [==============================] - 17s 1ms/step - loss: 0.4032 - accuracy: 0.8930
Epoch 5/200
14165/14165 [==============================] - 17s 1ms/step - loss: 0.3445 - accuracy: 0.8962
Epoch 6/200
14165/14165 [==============================] - 17s 1ms/step - loss: 0.3354 - accuracy: 0.8976
Epoch 7/200
14165/14165 [==============================] - 18s 1ms/step - loss: 0.3282 - accuracy: 0.8980
Epoch 8/200
14165/14165 [==============================] - 18s 1ms/step - loss: 0.3286 - accuracy: 0.8980
Epoch 9/200
14165/14165 [==============================] - 18s 1ms/step - loss: 0.3224 - accuracy: 0.8986
Epoch 10/200
14165/14165 [====================

14165/14165 [==============================] - 18s 1ms/step - loss: 0.1555 - accuracy: 0.9394
Epoch 154/200
14165/14165 [==============================] - 17s 1ms/step - loss: 0.1579 - accuracy: 0.9395
Epoch 155/200
14165/14165 [==============================] - 17s 1ms/step - loss: 0.1537 - accuracy: 0.9403
Epoch 156/200
14165/14165 [==============================] - 17s 1ms/step - loss: 0.1560 - accuracy: 0.9391
Epoch 157/200
14165/14165 [==============================] - 18s 1ms/step - loss: 0.1548 - accuracy: 0.9398
Epoch 158/200
14165/14165 [==============================] - 17s 1ms/step - loss: 0.1536 - accuracy: 0.9398
Epoch 159/200
14165/14165 [==============================] - 18s 1ms/step - loss: 0.1525 - accuracy: 0.9389
Epoch 160/200
14165/14165 [==============================] - 17s 1ms/step - loss: 0.1534 - accuracy: 0.9398
Epoch 161/200
14165/14165 [==============================] - 18s 1ms/step - loss: 0.1540 - accuracy: 0.9391
Epoch 162/200
14165/14165 [===============

(0.9285078904885843, 0.9346670576708294, 0.6344339622641509)

##### Alternative with Scaled Data

In [114]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train1 = scaler.transform(X_train)


scaler = StandardScaler()
scaler.fit(X_test)
X_test1 = scaler.transform(X_test)

In [115]:
def nn_model1(network, num_epoch, size):
    
    network.fit(X_train1, y_train, epochs = num_epoch, batch_size = size)
    nn_pred = network.predict_classes(X_test1)
    nn_pred_prob= network.predict(X_test1)


    print(confusion_matrix(y_test, nn_pred))
    print("Accuracy:",metrics.accuracy_score(y_test, nn_pred))
    print ("AUC Score:", roc_auc_score(y_test, nn_pred))
    print ("AUC Score prob:", roc_auc_score(y_test, nn_pred_prob))
    print ("F1 Score:", f1_score(y_test, nn_pred))    
    
    return metrics.accuracy_score(y_test, nn_pred),roc_auc_score(y_test, nn_pred_prob),f1_score(y_test, nn_pred)

In [116]:
network = models.Sequential()
network.add(layers.Dense(120, input_dim=15, activation='selu'))
network.add(layers.Dense(250,  activation='selu'))
network.add(layers.Dense(150, activation='selu'))
network.add(layers.Dense(1, activation='sigmoid'))
opt = optimizers.Adam(learning_rate=0.0001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-07)
network.compile(optimizer=opt,
loss='binary_crossentropy',
metrics=['accuracy'])

nn_model1(network, 200, 5)

Epoch 1/200
14165/14165 [==============================] - 17s 1ms/step - loss: 0.3327 - accuracy: 0.8948
Epoch 2/200
14165/14165 [==============================] - 16s 1ms/step - loss: 0.3221 - accuracy: 0.8964
Epoch 3/200
14165/14165 [==============================] - 16s 1ms/step - loss: 0.3186 - accuracy: 0.8966
Epoch 4/200
14165/14165 [==============================] - 16s 1ms/step - loss: 0.3161 - accuracy: 0.8968
Epoch 5/200
14165/14165 [==============================] - 16s 1ms/step - loss: 0.3139 - accuracy: 0.8970
Epoch 6/200
14165/14165 [==============================] - 16s 1ms/step - loss: 0.3122 - accuracy: 0.8968
Epoch 7/200
14165/14165 [==============================] - 15s 1ms/step - loss: 0.3102 - accuracy: 0.8970
Epoch 8/200
14165/14165 [==============================] - 15s 1ms/step - loss: 0.3080 - accuracy: 0.8971
Epoch 9/200
14165/14165 [==============================] - 15s 1ms/step - loss: 0.3057 - accuracy: 0.8975
Epoch 10/200
14165/14165 [====================

14165/14165 [==============================] - 17s 1ms/step - loss: 0.1572 - accuracy: 0.9386
Epoch 154/200
14165/14165 [==============================] - 18s 1ms/step - loss: 0.1581 - accuracy: 0.9375
Epoch 155/200
14165/14165 [==============================] - 17s 1ms/step - loss: 0.1566 - accuracy: 0.9384
Epoch 156/200
14165/14165 [==============================] - 17s 1ms/step - loss: 0.1560 - accuracy: 0.9392
Epoch 157/200
14165/14165 [==============================] - 18s 1ms/step - loss: 0.1555 - accuracy: 0.9392
Epoch 158/200
14165/14165 [==============================] - 17s 1ms/step - loss: 0.1551 - accuracy: 0.9387
Epoch 159/200
14165/14165 [==============================] - 17s 1ms/step - loss: 0.1551 - accuracy: 0.9392
Epoch 160/200
14165/14165 [==============================] - 17s 1ms/step - loss: 0.1542 - accuracy: 0.9393
Epoch 161/200
14165/14165 [==============================] - 18s 1ms/step - loss: 0.1539 - accuracy: 0.9396
Epoch 162/200
14165/14165 [===============

(0.9293644779758179, 0.9321850091184541, 0.6038433111603844)